In [0]:
%run ./Silver

In [0]:
class WorkFlowRunner:
    def __init__(self,name):
        self.name=name
    def runner(self):
        if name=="TotalRevenueWorkflow":
            firstworkFlow = TotalRevenueWorkflow()
            return firstworkFlow.runner()
        elif name=="ProductRevenueWorkflow":
            secondworkFlow = ProductRevenueWorkflow()
            return secondworkFlow.runner()
        elif name=="TopSellingWorkflow":
            thirdworkFlow = TopSellingWorkflow()
            return thirdworkFlow.runner()
        elif name=="TimelyRevenueWorkflow":
            fourthworkFlow = TimelyRevenueWorkflow()
            return fourthworkFlow.runner()
        elif name=="GeoRevenueWorkflow":
            fifthworkFlow = GeoRevenueWorkflow()
            return fifthworkFlow.runner()
        elif name=="CustomerValueWorkflow":
            sixthworkFlow = CustomerValueWorkflow()
            return sixthworkFlow.runner()

In [0]:
class TotalRevenueWorkflow:
    def __init__(self):
        pass
    def runner(self):
        #step 1 extract required data from different sources
        extractor = Order_items()
        order_items_Df = extractor.extract()

        #step 2 implement the transformation logic
        #customers who have bought airpods after buying iphones
        preprocessed_order_items_df = get_preprocessed_order_items(order_items_Df)
        TotalRevenue = TotalRevenueTransformer()
        TotalRevenueTransformerDf = TotalRevenue.transform(preprocessed_order_items_df)
        display(TotalRevenueTransformerDf)

        TotalRevenueTransformerDf.write.format("delta").mode("overwrite").save("abfss://gold@ecommerceproject.dfs.core.windows.net/Result/total_revenue/delta_table")

        spark.sql("create table if not exists eco.gold.total_revenue location 'abfss://gold@ecommerceproject.dfs.core.windows.net/Result/total_revenue/delta_table'")
        
        gold_df = spark.table("eco.gold.total_revenue")
        display(gold_df)


In [0]:
eco.gold.revenue_trendeco.gold.total_revenueeco.gold.total_revenueeco.gold.total_revenuename = "TotalRevenueWorkflow"

workFlowRunner = WorkFlowRunner(name)
workFlowRunner.runner()

Databricks visualization. Run in Databricks to view.

In [0]:
class ProductRevenueWorkflow:
    def __init__(self):
        pass
    def runner(self):
        #Extraction
        extractor = Order_items()
        orderitemDf = extractor.extract()
        extractor = Products_Category()
        productcatDf = extractor.extract()
        extractor = Products()
        productDf = extractor.extract()

        #Tranformation: Preprocessing
        preprocessed_products_df = get_preprocessed_products(productDf)
        preprocessed_product_category_df = get_preprocessed_product_category(productcatDf)
        preprocessed_order_items_df = get_preprocessed_order_items(orderitemDf)

        #Tranformation: Joins
        joined_df = productwise_revenue_join(preprocessed_order_items_df,preprocessed_products_df,preprocessed_product_category_df)

        #Transformation: Aggregation
        revenue_by_product_transformer = RevenueByProductCategoryTransformer()
        revenue_result = revenue_by_product_transformer.transform(joined_df)
        display(revenue_result)

        revenue_result.write.format("delta").mode("overwrite").save("abfss://gold@ecommerceproject.dfs.core.windows.net/Result/product_revenue/delta_table")

        spark.sql("create table if not exists eco.gold.product_revenue location 'abfss://gold@ecommerceproject.dfs.core.windows.net/Result/product_revenue/delta_table'")
        
        gold_df = spark.table("eco.gold.product_revenue")
        display(gold_df)


In [0]:
name = "ProductRevenueWorkflow"

workFlowRunner = WorkFlowRunner(name)
workFlowRunner.runner()

Databricks visualization. Run in Databricks to view.

In [0]:
class TopSellingWorkflow:
    def __init__(self):
        pass
    def runner(self):
        #Extraction
        extractor = Order_items()
        inputDf = extractor.extract()

        #Tranformation: Preprocessing
        preprocessed_order_items_df = get_preprocessed_order_items(inputDf)
        
        #Transformation: Aggregation
        top_selling_transformer = TopSellingProduct()
        top_selling_result = top_selling_transformer.transform(preprocessed_order_items_df)
        display(top_selling_result)

        top_selling_result.write.format("delta").mode("overwrite").save("abfss://gold@ecommerceproject.dfs.core.windows.net/Result/top_product/delta_table")

        spark.sql("create table if not exists eco.gold.top_product location 'abfss://gold@ecommerceproject.dfs.core.windows.net/Result/top_product/delta_table'")
        
        gold_df = spark.table("eco.gold.top_product")
        display(gold_df)        

In [0]:
name = "TopSellingWorkflow"

workFlowRunner = WorkFlowRunner(name)
workFlowRunner.runner()

In [0]:
source_path = "abfss://stream-data@ecommerceproject.dfs.core.windows.net/"
bronze_table_path = "abfss://bronze@ecommerceproject.dfs.core.windows.net/orders"
delta_table_path="abfss://bronze@ecommerceproject.dfs.core.windows.net/orders/delta_table/"
silver_table_path = "abfss://silver@ecommerceproject.dfs.core.windows.net/orders/"

In [0]:
class TimelyRevenueWorkflow:
    def __init__(self):
        pass
    def runner(self):
        #Extraction
        extractor = Order_items()
        orderitemDf = extractor.extract()

        extractor = StreamingProcessor(source_path, bronze_table_path)
        orderDf = extractor.start_streaming_job()
        orderDf.awaitTermination(30)

        #Tranformation: Preprocessing & Joins
        preprocessed_order_items_df = get_preprocessed_order_items(orderitemDf)

        preprocessed_order_df=IncrementalPreProcessor(delta_table_path, silver_table_path)
        query=preprocessed_order_df.preprocess_stream()
        query.awaitTermination(30)
        preprocessed_orders_df=spark.readStream.format("delta").load(silver_table_path)

        #Transformation:Join
        joined_df = order_items_join(preprocessed_orders_df,preprocessed_order_items_df)

        #Transformation: Aggregation
        revenue_by_time_transformer = RevenueTrendOverTimeTransformer()
        revenue_result = revenue_by_time_transformer.transform(joined_df)
        display(revenue_result)
        # Start writing the transformation output to another table or sink
        query = (revenue_result.writeStream
                 .format("delta")
                 .outputMode("complete")  # Or "complete" if doing aggregations
                 .option("checkpointLocation", "abfss://gold@ecommerceproject.dfs.core.windows.net/Result/revenue_trend/checkpoint")
                 .start("abfss://gold@ecommerceproject.dfs.core.windows.net/Result/revenue_trend/delta_table"))
        query.awaitTermination(15)
        spark.sql("create table if not exists eco.gold.revenue_trend location 'abfss://gold@ecommerceproject.dfs.core.windows.net/Result/revenue_trend/delta_table'")
        gold_stream_df = spark.readStream.format("delta").load("abfss://gold@ecommerceproject.dfs.core.windows.net/Result/revenue_trend/delta_table")
        display(gold_stream_df)

In [0]:
name = "TimelyRevenueWorkflow"

workFlowRunner = WorkFlowRunner(name)
workFlowRunner.runner()

In [0]:
class GeoRevenueWorkflow:
    def __init__(self):
        pass
    def runner(self):
        #Extraction
        orderi_extractor = Order_items()
        orderitemDf = orderi_extractor.extract()
        cust_extractor=Customers()
        customerDf = cust_extractor.extract()
        geo_extractor=Geolocation()
        geoDf=geo_extractor.extract()
        extractor = StreamingProcessor(source_path, bronze_table_path)
        orderDf = extractor.start_streaming_job()
        orderDf.awaitTermination(30)

        #Tranformation: Preprocessing & Joins
        preprocessed_order_items_df = get_preprocessed_order_items(orderitemDf)
        preprocessed_customer_df = get_preprocessed_customers(customerDf)
        preprocessed_geo_df=get_preprocessed_geolocation(geoDf)
        preprocessed_order_df=IncrementalPreProcessor(delta_table_path, silver_table_path)
        query=preprocessed_order_df.preprocess_stream()
        query.awaitTermination(30)
        preprocessed_order_df=spark.readStream.format("delta").load(silver_table_path)

        #Transformation:Join
        joined_df = geographic_revenue_join(preprocessed_order_df,preprocessed_order_items_df,preprocessed_customer_df,preprocessed_geo_df)
        # stream_joindf=spark.readStream.format("delta").load("abfss://gold@ecomadls.dfs.core.windows.net/geographic_revenue/delta_table")

        #Transformation: Aggregation
        revenue_by_state_transformer = GeographicRevenueTransformer()
        revenue_result = revenue_by_state_transformer.transform(joined_df)
        display(revenue_result)
        query = (revenue_result.writeStream
                 .format("delta")
                 .outputMode("complete")  # Or "complete" if doing aggregations
                 .option("checkpointLocation", "abfss://gold@ecommerceproject.dfs.core.windows.net/Result/geographic_revenue/checkpoint")
                 .start("abfss://gold@ecommerceproject.dfs.core.windows.net/Result/geographic_revenue/delta_table"))
        query.awaitTermination(15)
        spark.sql("create table if not exists eco.gold.geographic_revenue location 'abfss://gold@ecommerceproject.dfs.core.windows.net/Result/geographic_revenue/delta_table'")
        gold_stream_df = spark.readStream.format("delta").load("abfss://gold@ecommerceproject.dfs.core.windows.net/Result/geographic_revenue/delta_table")
        display(gold_stream_df)
        

In [0]:
name = "GeoRevenueWorkflow"

workFlowRunner = WorkFlowRunner(name)
workFlowRunner.runner()

In [0]:
class CustomerValueWorkflow:
    def __init__(self):
        pass
    def runner(self):
        #Extraction
        orderi_extractor = Order_items()
        orderitemDf = orderi_extractor.extract()

        cust_extractor=Customers()
        customerDf = cust_extractor.extract()

        extractor = StreamingProcessor(source_path, bronze_table_path)
        orderDf = extractor.start_streaming_job()
        orderDf.awaitTermination(30)

        #Tranformation: Preprocessing & Joins
        preprocessed_order_items_df = get_preprocessed_order_items(orderitemDf)
        preprocessed_customer_df = get_preprocessed_customers(customerDf)
        preprocessed_order_df=IncrementalPreProcessor(delta_table_path, silver_table_path)
        query=preprocessed_order_df.preprocess_stream()
        query.awaitTermination(30)
        preprocessed_order_df=spark.readStream.format("delta").load(silver_table_path)

        #Transformation:Join
        joined_df = customer_spending_join(preprocessed_order_df,preprocessed_order_items_df,preprocessed_customer_df)
        # stream_joindf=spark.readStream.format("delta").load("abfss://gold@ecomadls.dfs.core.windows.net/customer_spending/delta_table")

        #Transformation: Aggregation
        customer_value_transformer = CustomerValueTransformer()
        revenue_result = customer_value_transformer.transform(joined_df)
        display(revenue_result)
        query = (revenue_result.writeStream
                 .format("delta")
                 .outputMode("complete")  
                 .option("checkpointLocation", "abfss://gold@ecommerceproject.dfs.core.windows.net/Result/customer_value/checkpoint")
                 .start("abfss://gold@ecommerceproject.dfs.core.windows.net/Result/customer_value/delta_table"))
        query.awaitTermination(15)
        spark.sql("create table eco.gold.customer_value location 'abfss://gold@ecommerceproject.dfs.core.windows.net/Result/customer_value/delta_table'")
        gold_stream_df = spark.readStream.format("delta").load("abfss://gold@ecommerceproject.dfs.core.windows.net/Result/customer_value/delta_table")
        display(gold_stream_df)
        

In [0]:
name = "CustomerValueWorkflow"

workFlowRunner = WorkFlowRunner(name)
workFlowRunner.runner()